In [1]:
import os
import shutil

import pandas as pd
from rapidfuzz import fuzz
from datetime import datetime, timezone

from helpers import log, check_file_exists, load_csv, save_csv, remove_file_or_dir, filter_directory

In [2]:
# In the understat folder we have some useful data about previous fixtures, let's add it to fixtures.csv


In [3]:
def check_for_duplicate_players_by_name(data_directory, seasons):
    """
    Check for duplicate player names across multiple seasons.

    Args:
        data_directory (str): Path to the base data directory.
        seasons (list): List of season names to check (e.g., ["2020-21", "2021-22"]).

    Returns:
        dict: A dictionary where keys are season names and values are DataFrames
              of duplicate player names with ids.
    """
    duplicate_players = {}
    
    for season in seasons:
        season_path = os.path.join(data_directory, season)
        player_idlist_path = os.path.join(season_path, "player_idlist.csv")
        
        if not check_file_exists(player_idlist_path):
            log(f"player_idlist.csv not found for season {season}. Skipping.", level="WARNING")
            continue

        player_idlist = load_csv(player_idlist_path)
        if player_idlist is None:
            continue

        duplicates = player_idlist.duplicated(subset=['first_name', 'second_name'], keep=False)
        duplicate_names = player_idlist.loc[duplicates, ['id', 'first_name', 'second_name']].drop_duplicates()
        
        if not duplicate_names.empty:
            duplicate_players[season] = duplicate_names
            log(f"Found duplicate names in season {season}:")
            for _, row in duplicate_names.iterrows():
                log(f"    ID: {row['id']} - {row['first_name']} {row['second_name']}", level="INFO")
        else:
            log(f"No duplicate names found in season {season}.")
            
    return duplicate_players

data_directory = "Fantasy-Premier-League/data"
seasons = ["2022-23", "2023-24", "2024-25"]

duplicates_by_name = check_for_duplicate_players_by_name(data_directory, seasons)

INFO: Found duplicate names in season 2022-23:
INFO:     ID: 499 - Ben Davies
INFO:     ID: 432 - Ben Davies
INFO: No duplicate names found in season 2023-24.
INFO: No duplicate names found in season 2024-25.


In [4]:
# Here we get rid of the Ben Davies that's no longer in the game
def remove_players_and_folders(data_directory, players_to_remove):
    """
    Remove players from player_idlist.csv and their associated folders.

    Args:
        data_directory (str): Base directory containing the season data.
        players_to_remove (list): List of dictionaries with player details to remove.

    Example Player Format:
        {"season": "2020-21", "first_name": "Ben", "second_name": "Davies", "id": 653}
    """
    for player in players_to_remove:
            season = player["season"]
            player_id = player["id"]
            first_name = player["first_name"]
            second_name = player["second_name"]

            season_path = os.path.join(data_directory, season)
            player_idlist_path = os.path.join(season_path, "player_idlist.csv")
            players_folder_path = os.path.join(season_path, "players")
            player_folder_name = f"{first_name}_{second_name}_{player_id}"
            player_folder_path = os.path.join(players_folder_path, player_folder_name)

            if not check_file_exists(player_idlist_path):
                log(f"player_idlist.csv not found for season {season}. Skipping.", level="WARNING")
                continue
                
            player_idlist = load_csv(player_idlist_path)
            if player_idlist is None:
                continue

            matching_rows = player_idlist[
                (player_idlist["id"] == player_id) &
                (player_idlist["first_name"] == first_name) &
                (player_idlist["second_name"] == second_name)
            ]

            if not matching_rows.empty:
                player_idlist.drop(matching_rows.index, inplace=True)
                save_csv(player_idlist, player_idlist_path)
                log(f"Removed {first_name} {second_name} (ID: {player_id}) from {player_idlist_path}.")
            else:
                log(f"Player {first_name} {second_name} (ID: {player_id}) not found in {player_idlist_path}. Skipping.", level="WARNING")

            remove_file_or_dir(player_folder_path)

data_directory = "Fantasy-Premier-League/data"
players_to_remove = [
        {"season": "2022-23", "first_name": "Ben", "second_name": "Davies", "id": 499},
]

remove_players_and_folders(data_directory, players_to_remove)

INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/player_idlist.csv
INFO: Removed Ben Davies (ID: 499) from Fantasy-Premier-League/data/2022-23/player_idlist.csv.
INFO: Deleted directory: Fantasy-Premier-League/data/2022-23/players/Ben_Davies_499


In [5]:
# First let's get each player position and id for the season
def process_season_data(data_directory, seasons, output_folder_name="processed_data"):
    """
    Process player data for each season by merging player ID lists with cleaned data.

    Args:
        data_directory (str): Base directory containing the season data.
        seasons (list): List of season names to process (e.g., ["2020-21", "2021-22"]).
        output_folder_name (str): Name of the folder to store the processed files.
    Returns:
        None
    """
    for season in seasons:
        season_path = os.path.join(data_directory, season)
        player_idlist_path = os.path.join(season_path, "player_idlist.csv")
        cleaned_players_path = os.path.join(season_path, "cleaned_players.csv")

        output_folder_path = os.path.join(season_path, output_folder_name)
        os.makedirs(output_folder_path, exist_ok=True)

        output_file_path = os.path.join(output_folder_path, "processed_players.csv")
        
        if not os.path.isdir(season_path):
            log(f"Season directory {season_path} does not exist. Skipping.", level="WARNING")
            continue

        if not (check_file_exists(player_idlist_path) and check_file_exists(cleaned_players_path)):
            log(f"Missing required files in {season_path}: player_idlist.csv or cleaned_players.csv", level="WARNING")
            continue
        
        player_idlist_df = load_csv(player_idlist_path)
        cleaned_players_df = load_csv(cleaned_players_path)

        if player_idlist_df is None or cleaned_players_df is None:
            continue

        try:
            # Merge the datasets
            merged_df = pd.merge(
                player_idlist_df[['id', 'first_name', 'second_name']],
                cleaned_players_df[['first_name', 'second_name', 'element_type']],
                on=['first_name', 'second_name'],
                how='inner'
            )

            merged_df.rename(columns={'element_type': 'position'}, inplace=True)

            save_csv(merged_df, output_file_path)
            log(f"Processed and saved: {output_file_path}")
        except Exception as e:
            log(f"Error processing season {season}: {e}", level="ERROR")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2024-25", "2023-24", "2022-23"]

process_season_data(data_directory, seasons)

INFO: Saved CSV: Fantasy-Premier-League/data/2024-25/processed_data/processed_players.csv
INFO: Processed and saved: Fantasy-Premier-League/data/2024-25/processed_data/processed_players.csv
INFO: Saved CSV: Fantasy-Premier-League/data/2023-24/processed_data/processed_players.csv
INFO: Processed and saved: Fantasy-Premier-League/data/2023-24/processed_data/processed_players.csv
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/processed_data/processed_players.csv
INFO: Processed and saved: Fantasy-Premier-League/data/2022-23/processed_data/processed_players.csv


In [6]:
# This is used to create a new file with player ids across seasons
def merge_player_ids(data_directory, seasons, output_file="master_player_list.csv"):
    """
    Merge player IDs across seasons into a consolidated master list.

    Args:
        data_directory (str): Base directory containing the season data.
        seasons (list): List of season names to process (e.g., ["2020-21", "2021-22"]).
        output_file (str): Name of the output file for the master player list.

    Returns:
        pd.DataFrame: The consolidated DataFrame of player IDs.
    """
    player_data = {}
    next_unique_id = 1
    main_season = "2024-25"
    main_season_path = os.path.join(data_directory, main_season, "processed_data", "processed_players.csv")

    # Process the main season first. We want to keep these id's since we will use same ones for predictions. 
    if check_file_exists(main_season_path):
        main_processed_df = load_csv(main_season_path)
        if main_processed_df is not None:
            for _, row in main_processed_df.iterrows():
                full_name = f"{row['first_name']} {row['second_name']}"
                if full_name not in player_data:
                    player_data[full_name] = {
                        "First_Name": row['first_name'],
                        "Last_Name": row['second_name'],
                        "Unique_ID": row['id'],
                        "24_id": row['id']
                    }
                next_unique_id = max(next_unique_id, row['id'] + 1)
    else:
        log(f"Missing processed_players.csv for main season: {main_season}", level="WARNING")

    # Process all other seasons
    for season in seasons:
        if season == main_season:
            continue

        season_path = os.path.join(data_directory, season, "processed_data", "processed_players.csv")
        if check_file_exists(season_path):
            processed_df = load_csv(season_path)
            if processed_df is not None:
                season_short = season[:4][-2:]
                for _, row in processed_df.iterrows():
                    full_name = f"{row['first_name']} {row['second_name']}"
                    if full_name not in player_data:
                        player_data[full_name] = {
                            "First_Name": row['first_name'],
                            "Last_Name": row['second_name'],
                            "Unique_ID": next_unique_id
                        }
                        next_unique_id += 1
                    # Add season-specific ID
                    player_data[full_name][f"{season_short}_id"] = row['id']
        else:
            log(f"Missing processed_players.csv for season: {season}", level="WARNING")

    # Convert player data to DataFrame
    consolidated_df = pd.DataFrame.from_dict(player_data, orient='index').reset_index(drop=True)

    # Ensure all ID columns are integers
    id_columns = [col for col in consolidated_df.columns if col.endswith("_id")]
    consolidated_df[id_columns] = consolidated_df[id_columns].fillna(-1).astype(int)

    season_columns = sorted(
        [col for col in consolidated_df.columns if col.endswith("_id")],
        reverse=True
    )
    # Reorder columns
    other_columns = [col for col in consolidated_df.columns if col not in season_columns]
    consolidated_df = consolidated_df[other_columns + season_columns]
    
    # Save the consolidated DataFrame
    output_path = os.path.join(data_directory, output_file)
    save_csv(consolidated_df, output_path)
    log(f"Consolidated player data saved to {output_path}")

    return consolidated_df


data_directory = "Fantasy-Premier-League/data"
seasons = ["2022-23", "2023-24", "2024-25"]

master_player_list = merge_player_ids(data_directory, seasons, output_file="master_player_v1.csv")

INFO: Saved CSV: Fantasy-Premier-League/data/master_player_v1.csv
INFO: Consolidated player data saved to Fantasy-Premier-League/data/master_player_v1.csv


In [7]:
# # Script below was used to get duplicate candidates (players can have name changes across seasons). After finding them I checked it manually

# consolidated_file = "master_player_v1.csv"
# consolidated_path = os.path.join(data_directory, consolidated_file)
# consolidated_df = pd.read_csv(consolidated_path)

# SIMILARITY_THRESHOLD = 80
# FIRST_NAME_SIMILARITY_THRESHOLD = 40

# def is_pair(player_a, player_b, id_columns):
#     # Check for ID overlaps across seasons
#     id_overlap = any(
#         player_a[col] != -1 and player_b[col] != -1
#         for col in id_columns
#     )
#     if id_overlap:
#         return False
    
#     # Now we check the names similarity
#     first_name_similarity = fuzz.partial_ratio(player_a["First_Name"], player_b["First_Name"])
#     if first_name_similarity <= FIRST_NAME_SIMILARITY_THRESHOLD:
#         return False
    
#     # Check if one last name is contained in the other
#     if player_a["Last_Name"] in player_b["Last_Name"] or player_b["Last_Name"] in player_a["Last_Name"]:
#         return True

#     # Calculate string similarity only if containment check fails
#     similarity = fuzz.partial_ratio(player_a["Last_Name"], player_b["Last_Name"])
#     return similarity >= SIMILARITY_THRESHOLD

# # Identify all ID columns
# id_columns = [col for col in consolidated_df.columns if col.endswith("_id")]
# pairs = []
# for i, player_a in consolidated_df.iterrows():
#     for j, player_b in consolidated_df.iterrows():
#         if i >= j:  # Avoid duplicate and self-pairs
#             continue
#         if is_pair(player_a, player_b, id_columns):
#             pairs.append({
#                 "Player_A_First_Name": player_a["First_Name"],
#                 "Player_A_Last_Name": player_a["Last_Name"],
#                 "Player_B_First_Name": player_b["First_Name"],
#                 "Player_B_Last_Name": player_b["Last_Name"],
#             })

# # Convert pairs to a DataFrame
# created_csv_dir = "created_csv"
# pairs_df = pd.DataFrame(pairs)
# if not pairs_df.empty:
#     output_path = os.path.join(created_csv_dir, "players_with_potential_renames.csv")
#     pairs_df.to_csv(output_path, index=False)
#     print(f"Potential renamed players saved to {output_path}")
# else:
#     print("No potential renamed players found.")

In [8]:
# After manual checking the candidates now we have to update master_player_list.csv 
# (new file will me master_player_verified).
# We might get some warnings here but we can safely ignore them (one player can have multiple rows and deleting one of them causes errors
# but we still merge the data to the one with lowest ID.
def update_master_player_list(data_directory, master_player_file, verified_renames_file, output_file="master_player_v2.csv"):
    """
    Update the master player list by merging duplicate player data (same players with different names across seasons)
    and saving the verified list.

    Args:
        data_directory (str): Base directory containing the data.
        master_player_file (str): Name of the master player list file (e.g., "master_player_list.csv").
        verified_renames_file (str): Path to the verified renames CSV file.
        output_file (str): Name of the output file for the updated master list.

    Returns:
        None
    """
    master_player_path = os.path.join(data_directory, master_player_file)
    output_path = os.path.join(data_directory, output_file)

    # Load the verified renames and master player list
    verified_renames = load_csv(verified_renames_file)
    master_player_list = load_csv(master_player_path)

    if verified_renames is None or master_player_list is None:
        log("Failed to load required files. Exiting update process.", level="ERROR")
        return

    # Iterate through verified renames and process each pair. The way finding duplicate candidates work player_b will have greater
    # ID than player_a
    for _, row in verified_renames.iterrows():
        player_a_first = row["Player_A_First_Name"]
        player_a_last = row["Player_A_Last_Name"]
        player_b_first = row["Player_B_First_Name"]
        player_b_last = row["Player_B_Last_Name"]

        # Find rows for Player A and Player B
        player_a_row = master_player_list[
            (master_player_list["First_Name"] == player_a_first) &
            (master_player_list["Last_Name"] == player_a_last)
        ]

        player_b_row = master_player_list[
            (master_player_list["First_Name"] == player_b_first) &
            (master_player_list["Last_Name"] == player_b_last)
        ]

        # Check for missing players
        if player_a_row.empty or player_b_row.empty:
            log(f"Warning: Missing player in master_player_list. Player A: {player_a_first} {player_a_last}, "
                f"Player B: {player_b_first} {player_b_last}", level="INFO")
            continue

        player_a_id = player_a_row["Unique_ID"].iloc[0]
        player_b_id = player_b_row["Unique_ID"].iloc[0]

        # Update the smaller ID row with non -1 season data from the larger ID row
        for season in ["24_id", "23_id", "22_id"]:
            if player_b_row[season].iloc[0] != -1:
                master_player_list.loc[master_player_list["Unique_ID"] == player_a_id, season] = player_b_row[season].iloc[0]

        # Remove Player B
        master_player_list = master_player_list[master_player_list["Unique_ID"] != player_b_id]
        log(f"Merged Player B (ID: {player_b_id}) into Player A (ID: {player_a_id}). Removed Player B.", level="INFO")

    # Save the updated master player list
    save_csv(master_player_list, output_path)
    log(f"Updated master player list saved to {output_path}")

data_directory = "Fantasy-Premier-League/data"
master_player_file = "master_player_v1.csv"
verified_renames_file = "created_csv/verified_renames_2022.csv"

update_master_player_list(data_directory, master_player_file, verified_renames_file)

INFO: Merged Player B (ID: 714) into Player A (ID: 43). Removed Player B.
INFO: Merged Player B (ID: 700) into Player A (ID: 56). Removed Player B.
INFO: Merged Player B (ID: 711) into Player A (ID: 569). Removed Player B.
INFO: Merged Player B (ID: 893) into Player A (ID: 80). Removed Player B.
INFO: Merged Player B (ID: 741) into Player A (ID: 680). Removed Player B.
INFO: Merged Player B (ID: 1082) into Player A (ID: 97). Removed Player B.
INFO: Merged Player B (ID: 757) into Player A (ID: 111). Removed Player B.
INFO: Merged Player B (ID: 1076) into Player A (ID: 111). Removed Player B.
INFO: Merged Player B (ID: 807) into Player A (ID: 221). Removed Player B.
INFO: Merged Player B (ID: 805) into Player A (ID: 624). Removed Player B.
INFO: Merged Player B (ID: 1051) into Player A (ID: 239). Removed Player B.
INFO: Merged Player B (ID: 850) into Player A (ID: 249). Removed Player B.
INFO: Merged Player B (ID: 784) into Player A (ID: 274). Removed Player B.
INFO: Merged Player B (ID:

In [9]:
# With the new list we can move on to renaming player folders to have the new ids and their positions
# We also update the processed_players

def update_processed_players_and_rename_folders(data_directory, seasons, master_player_file):
    """
    Update processed players' IDs and names, and rename player folders.

    Args:
        data_directory (str): Base directory containing the data.
        seasons (list): List of seasons to process (e.g., ["2020-21", "2021-22"]).
        master_player_file (str): Name of the master player list file (e.g., "master_player_verified.csv").

    Returns:
        None
    """
    master_player_path = os.path.join(data_directory, master_player_file)
    master_player_verified = load_csv(master_player_path)

    if master_player_verified is None:
        log(f"Failed to load {master_player_file}. Exiting process.", level="ERROR")
        return

    for season in seasons:
        season_path = os.path.join(data_directory, season)
        players_folder_path = os.path.join(season_path, "players")
        processed_file = os.path.join(season_path, "processed_data", "processed_players.csv")
        verified_file = os.path.join(season_path, "processed_data", "processed_players_v2.csv")

        # Special case for "2024-25". We just copy here.
        if season == "2024-25":
            log(f"Processing special case: {season}")
            if check_file_exists(processed_file):
                try:
                    shutil.copy(processed_file, verified_file)
                    log(f"Copied {processed_file} to {verified_file}")
                except Exception as e:
                    log(f"Error copying {processed_file} to {verified_file}: {e}", level="ERROR")
            else:
                log(f"Processed players file not found for season: {season}", level="WARNING")
            continue

        # Process other seasons
        season_short = season[:4][-2:]  # Extract short season ID (e.g., "21" for "2021-22")
        season_id_column = f"{season_short}_id"

        if not check_file_exists(processed_file):
            log(f"Processed players file not found for season: {season}", level="WARNING")
            continue

        processed_players = load_csv(processed_file)
        if processed_players is None:
            continue

        # Rename player folders and update processed players
        if check_file_exists(players_folder_path):
            player_folders = [
                f for f in os.listdir(players_folder_path)
                if os.path.isdir(os.path.join(players_folder_path, f))
            ]
        else:
            log(f"Players folder not found for season {season}. Skipping folder renames.", level="WARNING")
            player_folders = []

        for index, row in processed_players.iterrows():
            current_id = row["id"]
            matching_row = master_player_verified[master_player_verified[season_id_column] == current_id]

            if matching_row.empty:
                log(f"No matching row in master list for player ID {current_id} in season {season}.", level="WARNING")
                continue

            # Get new player data
            new_data = matching_row.iloc[0]
            new_unique_id = new_data["Unique_ID"]
            new_first_name = new_data["First_Name"]
            new_last_name = new_data["Last_Name"]

            # Update processed_players DataFrame
            processed_players.at[index, "id"] = new_unique_id
            processed_players.at[index, "first_name"] = new_first_name
            processed_players.at[index, "second_name"] = new_last_name

            # Rename player folder
            matching_folder = next(
                (f for f in player_folders if f.endswith(f"_{current_id}")), None
            )
            if matching_folder:
                old_path = os.path.join(players_folder_path, matching_folder)
                new_folder_name = f"{new_first_name}_{new_last_name}_{new_unique_id}"
                new_path = os.path.join(players_folder_path, new_folder_name)

                if not check_file_exists(new_path):  # Avoid overwriting existing folders
                    try:
                        shutil.move(old_path, new_path)
                        log(f"Renamed folder {matching_folder} to {new_folder_name}")
                        player_folders.remove(matching_folder)  # Remove processed folder
                    except Exception as e:
                        log(f"Error renaming {old_path} to {new_path}: {e}", level="ERROR")
                else:
                    log(f"Destination folder {new_path} already exists. Skipping rename.", level="WARNING")

        save_csv(processed_players, verified_file)
        log(f"Updated processed players saved for season {season} to {verified_file}")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2022-23", "2023-24", "2024-25"]
master_player_file = "master_player_v2.csv"
update_processed_players_and_rename_folders(data_directory, seasons, master_player_file)

INFO: Renamed folder Granit_Xhaka_3 to Granit_Xhaka_691
INFO: Renamed folder Mohamed_Elneny_4 to Mohamed_Elneny_692
INFO: Renamed folder Rob_Holding_5 to Rob_Holding_202
INFO: Renamed folder Thomas_Partey_6 to Thomas_Partey_20
INFO: Renamed folder Martin_Ødegaard_7 to Martin_Ødegaard_13
INFO: Renamed folder Kieran_Tierney_8 to Kieran_Tierney_21
INFO: Renamed folder Nicolas_Pépé_9 to Nicolas_Pépé_693
INFO: Renamed folder Benjamin_White_10 to Benjamin_White_24
INFO: Renamed folder Emile_Smith Rowe_12 to Emile_Smith Rowe_19
INFO: Renamed folder Bukayo_Saka_13 to Bukayo_Saka_17
INFO: Renamed folder Takehiro_Tomiyasu_14 to Takehiro_Tomiyasu_694
INFO: Renamed folder Aaron_Ramsdale_15 to Aaron_Ramsdale_14
INFO: Renamed folder Gabriel_dos Santos Magalhães_16 to Gabriel_dos Santos Magalhães_3
INFO: Renamed folder Nuno_Varela Tavares_17 to Nuno_Varela Tavares_695
INFO: Renamed folder Gabriel_Martinelli Silva_19 to Gabriel_Martinelli Silva_9
INFO: Renamed folder Pablo_Marí Villar_20 to Pablo_Marí

In [10]:
# Now we will start manipulating player's data. First let's add to each player row his position
# (1 - GK, 2 - DEF, 3 - MID, 4 - FWD) and add it to the folder's name as well
# Let's also add the Full Name of the player and his unique id.

def update_player_folders_and_gw_data(season_path):
    """
    Update player folders with positions, unique IDs, and full names. 
    Also, update gw.csv files with these attributes.

    Args:
        season_path (str): Path to the season folder.

    Returns:
        None
    """
    position_mapping = {"GK": 1, "DEF": 2, "MID": 3, "FWD": 4}
    
    processed_players_path = os.path.join(season_path, "processed_data", "processed_players_v2.csv")
    players_folder_path = os.path.join(season_path, "players")
    
    if not check_file_exists(processed_players_path):
        log(f"{processed_players_path} not found. Skipping season.", level="WARNING")
        return
    
    processed_players = load_csv(processed_players_path)
    if processed_players is None:
        log(f"Failed to load {processed_players_path}. Skipping season.", level="ERROR")
        return
    
    for _, row in processed_players.iterrows():
        player_id = row['id']
        first_name = row['first_name']
        second_name = row['second_name']
        position_str = row['position']
        
        pos_numeric = position_mapping.get(position_str, 0)  # Default to 0 if unknown
        if pos_numeric == 0:
            log(f"Unknown position for player {first_name} {second_name} (ID: {player_id}). Skipping.", level="WARNING")
            continue
        
        old_folder_name = f"{first_name}_{second_name}_{player_id}"
        old_folder_path = os.path.join(players_folder_path, old_folder_name)
        if not check_file_exists(old_folder_path):
            log(f"Folder {old_folder_name} not found. Skipping.", level="WARNING")
            continue

        new_folder_name = f"{position_str}_{old_folder_name}"
        new_folder_path = os.path.join(players_folder_path, new_folder_name)
        
        try:
            if not check_file_exists(new_folder_path):  # Avoid overwriting existing folders
                shutil.move(old_folder_path, new_folder_path)
                log(f"Renamed folder: {old_folder_name} -> {new_folder_name}", level="INFO")
            else:
                log(f"Destination folder {new_folder_name} already exists. Skipping rename.", level="WARNING")
        except Exception as e:
            log(f"Error renaming {old_folder_name} to {new_folder_name}: {e}", level="ERROR")
            continue
        
        gw_file_path = os.path.join(new_folder_path, "gw.csv")
        
        if check_file_exists(gw_file_path):
            try:
                gw_data = load_csv(gw_file_path)
                if gw_data is not None:
                    # Update element to unique_id and rename column
                    if 'element' in gw_data.columns:
                        gw_data.rename(columns={'element': 'unique_id'}, inplace=True)
                        gw_data['unique_id'] = player_id
                    
                    # Add POS and Full Name
                    gw_data['POS'] = pos_numeric
                    gw_data['Full Name'] = f"{first_name} {second_name}"
                    
                    # Save updated gw.csv
                    save_csv(gw_data, gw_file_path)
                    log(f"Updated POS and unique_id in {gw_file_path}", level="INFO")
            except Exception as e:
                log(f"Error updating {gw_file_path}: {e}", level="ERROR")
        else:
            log(f"{gw_file_path} not found. Skipping GW data update.", level="WARNING")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2022-23", "2023-24", "2024-25"]

for season in seasons:
    season_path = os.path.join(data_directory, season)
    log(f"Processing season: {season_path}", level="INFO")
    update_player_folders_and_gw_data(season_path)

INFO: Processing season: Fantasy-Premier-League/data/2022-23
INFO: Renamed folder: Granit_Xhaka_691 -> MID_Granit_Xhaka_691
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Granit_Xhaka_691/gw.csv
INFO: Updated POS and unique_id in Fantasy-Premier-League/data/2022-23/players/MID_Granit_Xhaka_691/gw.csv
INFO: Renamed folder: Mohamed_Elneny_692 -> MID_Mohamed_Elneny_692
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Mohamed_Elneny_692/gw.csv
INFO: Updated POS and unique_id in Fantasy-Premier-League/data/2022-23/players/MID_Mohamed_Elneny_692/gw.csv
INFO: Renamed folder: Rob_Holding_202 -> DEF_Rob_Holding_202
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/DEF_Rob_Holding_202/gw.csv
INFO: Updated POS and unique_id in Fantasy-Premier-League/data/2022-23/players/DEF_Rob_Holding_202/gw.csv
INFO: Renamed folder: Thomas_Partey_20 -> MID_Thomas_Partey_20
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Thomas_Partey_20/gw.csv
INFO: 

In [11]:
# Now let's get a list of team id's from each season, similarly as we did we players
def generate_master_team_list(data_directory, seasons, output_file="master_team_list_v2.csv"):
    """
    Generate a master team list with unique IDs and seasonal IDs.

    Args:
        data_directory (str): Base directory containing the season data.
        seasons (list): List of season names (e.g., ["2023-24", "2022-23", ...]).
        output_file (str): Name of the output file for the master team list.

    Returns:
        pd.DataFrame: The consolidated DataFrame of team IDs across seasons.
    """
    team_data = {}
    next_unique_id = 1

    # Process 2024-25 season first to assign Unique IDs
    main_season = "2024-25"
    main_season_path = os.path.join(data_directory, main_season, "teams.csv")

    if check_file_exists(main_season_path):
        main_season_df = load_csv(main_season_path)
        if main_season_df is not None:
            for _, row in main_season_df.iterrows():
                team_name = row["name"]
                team_id = row["id"]
                if team_name not in team_data:
                    team_data[team_name] = {
                        "Name": team_name,
                        "Unique_Id": next_unique_id,
                        "24_Id": team_id,
                    }
                    next_unique_id += 1
    else:
        log(f"teams.csv not found for main season: {main_season}", level="ERROR")
        return None

    # Process other seasons
    for season in seasons:
        if season == main_season:
            continue

        season_short = season[:4][-2:]  # Extract year
        season_path = os.path.join(data_directory, season, "teams.csv")

        if check_file_exists(season_path):
            season_df = load_csv(season_path)
            if season_df is not None:
                for _, row in season_df.iterrows():
                    team_name = row["name"]
                    team_id = row["id"]
                    if team_name not in team_data:
                        # Add new team with placeholder for Unique_Id
                        team_data[team_name] = {"Name": team_name, "Unique_Id": next_unique_id}
                        next_unique_id += 1
                    team_data[team_name][f"{season_short}_Id"] = team_id
        else:
            log(f"Skipping season {season}. teams.csv not found.", level="WARNING")

    # Create DataFrame from team_data dictionary
    consolidated_df = pd.DataFrame.from_dict(team_data, orient="index")
    consolidated_df = consolidated_df.fillna(-1)  # Fill missing season IDs with -1
    consolidated_df = consolidated_df.astype({col: int for col in consolidated_df.columns if col.endswith("_Id")})

    output_path = os.path.join(data_directory, output_file)
    save_csv(consolidated_df, output_path)

    log(f"Master team list saved to {output_path}")
    return consolidated_df
    
data_directory = "Fantasy-Premier-League/data"
seasons = ["2024-25", "2023-24", "2022-23"]
generate_master_team_list(data_directory, seasons)

INFO: Saved CSV: Fantasy-Premier-League/data/master_team_list_v2.csv
INFO: Master team list saved to Fantasy-Premier-League/data/master_team_list_v2.csv


,Name,Unique_Id,24_Id,23_Id,22_Id
Arsenal,Arsenal,1,1,1,1
Aston Villa,Aston Villa,2,2,2,2
Bournemouth,Bournemouth,3,3,3,3
Brentford,Brentford,4,4,4,4
Brighton,Brighton,5,5,5,5
Chelsea,Chelsea,6,6,7,6
Crystal Palace,Crystal Palace,7,7,8,7
Everton,Everton,8,8,9,8
Fulham,Fulham,9,9,10,9
Ipswich,Ipswich,10,10,-1,-1


In [12]:
# Once we have that we have to go into each season folder (except 2024-25) and update:
# - the opponent_team in gw.csv for each player
# - team_a and team_h in fixtures.csv
# - team in players_raw.csv
# - id in teams.csv
def update_team_ids_in_seasons(data_directory, seasons, master_team_list_file):
    """
    Update team IDs in all relevant files across seasons based on the master team list.

    Args:
        data_directory (str): Base directory containing the season data.
        seasons (list): List of season names to update (e.g., ["2023-24", "2022-23"]).
        master_team_list_file (str): Path to the master team list CSV file.
    """

    master_team_list_path = os.path.join(data_directory, master_team_list_file)
    if not check_file_exists(master_team_list_path):
        log("Master team list file not found. Aborting updates.", level="ERROR")
        return

    master_team_list = load_csv(master_team_list_path)
    if master_team_list is None:
        log("Failed to load master team list. Aborting updates.", level="ERROR")
        return

    # Process each season
    for season in seasons:
        log(f"Processing season: {season}", level="INFO")
        season_path = os.path.join(data_directory, season)

        # Generate mapping for the current season: {this_season_team_id -> unique_id}
        season_short = season[:4][-2:]
        id_column = f"{season_short}_Id"
        if id_column not in master_team_list.columns:
            log(f"Column {id_column} not found in master team list. Skipping season {season}.", level="WARNING")
            continue
        
        id_mapping = (
            master_team_list.set_index(id_column)["Unique_Id"]
            .dropna()
            .to_dict()
        )
        id_mapping = {k: v for k, v in id_mapping.items() if k != -1}
        
        players_folder = os.path.join(season_path, "players")
        if check_file_exists(players_folder):
            for player_folder in os.listdir(players_folder):
                gw_file_path = os.path.join(players_folder, player_folder, "gw.csv")
                if check_file_exists(gw_file_path):
                    try:
                        gw_data = load_csv(gw_file_path)
                        if "opponent_team" in gw_data.columns:
                            gw_data["opponent_team"] = gw_data["opponent_team"].map(id_mapping)
                            save_csv(gw_data, gw_file_path)
                            log(f"Updated opponent_team in {gw_file_path}")
                    except Exception as e:
                        log(f"Error updating {gw_file_path}: {e}", level="ERROR")

        # Update fixtures.csv
        fixtures_file_path = os.path.join(season_path, "fixtures.csv")
        if check_file_exists(fixtures_file_path):
            try:
                fixtures_data = load_csv(fixtures_file_path)
                if "team_a" in fixtures_data.columns and "team_h" in fixtures_data.columns:
                    fixtures_data["team_a"] = fixtures_data["team_a"].map(id_mapping)
                    fixtures_data["team_h"] = fixtures_data["team_h"].map(id_mapping)
                    save_csv(fixtures_data, fixtures_file_path)
                    log(f"Updated team_a and team_h in {fixtures_file_path}")
            except Exception as e:
                log(f"Error updating {fixtures_file_path}: {e}", level="ERROR")

        # Update players_raw.csv
        players_raw_file_path = os.path.join(season_path, "players_raw.csv")
        if check_file_exists(players_raw_file_path):
            try:
                players_raw_data = load_csv(players_raw_file_path)
                if "team" in players_raw_data.columns:
                    players_raw_data["team"] = players_raw_data["team"].map(id_mapping)
                    save_csv(players_raw_data, players_raw_file_path)
                    log(f"Updated team in {players_raw_file_path}")
            except Exception as e:
                log(f"Error updating {players_raw_file_path}: {e}", level="ERROR")

        # Update teams.csv
        teams_file_path = os.path.join(season_path, "teams.csv")
        if check_file_exists(teams_file_path):
            try:
                teams_data = load_csv(teams_file_path)
                if "id" in teams_data.columns:
                    teams_data["id"] = teams_data["id"].map(id_mapping).fillna(teams_data["id"])
                    save_csv(teams_data, teams_file_path)
                    log(f"Updated id in {teams_file_path}")
            except Exception as e:
                log(f"Error updating {teams_file_path}: {e}", level="ERROR")

    log("All seasons updated successfully.", level="INFO")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2023-24", "2022-23"]  # Exclude "2024-25"
master_team_list_file = "master_team_list_v2.csv"

update_team_ids_in_seasons(data_directory, seasons, master_team_list_file)

INFO: Processing season: 2023-24
INFO: Saved CSV: Fantasy-Premier-League/data/2023-24/players/DEF_Kell_Watts_1223/gw.csv
INFO: Updated opponent_team in Fantasy-Premier-League/data/2023-24/players/DEF_Kell_Watts_1223/gw.csv
INFO: Saved CSV: Fantasy-Premier-League/data/2023-24/players/MID_Jacob_Ramsey_53/gw.csv
INFO: Updated opponent_team in Fantasy-Premier-League/data/2023-24/players/MID_Jacob_Ramsey_53/gw.csv
INFO: Saved CSV: Fantasy-Premier-League/data/2023-24/players/DEF_Joško_Gvardiol_350/gw.csv
INFO: Updated opponent_team in Fantasy-Premier-League/data/2023-24/players/DEF_Joško_Gvardiol_350/gw.csv
INFO: Saved CSV: Fantasy-Premier-League/data/2023-24/players/MID_Bradley_Ibrahim_1066/gw.csv
INFO: Updated opponent_team in Fantasy-Premier-League/data/2023-24/players/MID_Bradley_Ibrahim_1066/gw.csv
INFO: Saved CSV: Fantasy-Premier-League/data/2023-24/players/GK_Marek_Rodák_839/gw.csv
INFO: Updated opponent_team in Fantasy-Premier-League/data/2023-24/players/GK_Marek_Rodák_839/gw.csv
INF

In [13]:
# Player data doesn't have his own team id (and they can switch teams during season so we have to add that).
# For that we have to use fixtures.csv. Let's also rename round to gameweek
def add_own_team_to_players(data_directory, season, fixtures_file):
    """
    Adds the 'own_team' column to gw.csv files for each player and renames 'round' to 'gameweek'.

    Args:
        data_directory (str): Base directory containing the season data.
        season (str): Season name (e.g., "2022-23").
        fixtures_file (str): Name of the fixtures file (e.g., "fixtures.csv").

    Returns:
        None
    """
    fixtures_path = os.path.join(data_directory, season, fixtures_file)
    if not check_file_exists(fixtures_path):
        return

    fixtures = load_csv(fixtures_path)
    if fixtures is None:
        return

    players_folder = os.path.join(data_directory, season, "players")
    if not check_file_exists(players_folder):
        return

    for player_folder in os.listdir(players_folder):
        player_folder_path = os.path.join(players_folder, player_folder)
        gw_file_path = os.path.join(player_folder_path, "gw.csv")

        if check_file_exists(gw_file_path):
            try:
                gw_data = load_csv(gw_file_path)
                if gw_data is None:
                    continue

                # Ensure required columns exist
                if 'fixture' not in gw_data.columns or 'was_home' not in gw_data.columns:
                    log(f"Missing columns in {gw_file_path}. Skipping.", level="WARNING")
                    continue

                # Merge fixture data and compute 'own_team'
                gw_data = gw_data.merge(
                    fixtures[['id', 'team_a', 'team_h']],
                    left_on='fixture',
                    right_on='id',
                    how='left'
                )
                gw_data['own_team'] = gw_data.apply(
                    lambda x: x['team_h'] if x['was_home'] else x['team_a'], axis=1
                )

                # Rename 'round' to 'gameweek'
                if 'round' in gw_data.columns:
                    gw_data.rename(columns={'round': 'gameweek'}, inplace=True)

                # Drop unnecessary columns
                columns_to_drop = ['id', 'team_a', 'team_h']
                gw_data.drop(columns=[col for col in columns_to_drop if col in gw_data.columns], inplace=True)

                # Save the updated file
                save_csv(gw_data, gw_file_path)
                log(f"Updated {gw_file_path} with 'own_team' and 'gameweek'.")

            except Exception as e:
                log(f"Error processing {gw_file_path}: {e}", level="ERROR")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2022-23", "2023-24", "2024-25"]
fixtures_file = "fixtures.csv"

for season in seasons:
    log(f"Processing season: {season}", level="INFO")
    add_own_team_to_players(data_directory, season, fixtures_file)

INFO: Processing season: 2022-23
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Jacob_Ramsey_53/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/MID_Jacob_Ramsey_53/gw.csv with 'own_team' and 'gameweek'.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/GK_Marek_Rodák_839/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/GK_Marek_Rodák_839/gw.csv with 'own_team' and 'gameweek'.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/FWD_Ellis_Simms_826/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/FWD_Ellis_Simms_826/gw.csv with 'own_team' and 'gameweek'.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Sean_McAllister_829/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/MID_Sean_McAllister_829/gw.csv with 'own_team' and 'gameweek'.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/FWD_Julián_Álvarez_352/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/

Updated Fantasy-Premier-League/data/2020-21/players/MID_Fabio Henrique_Tavares_857/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Harrison_Reed_254/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ki-Jana_Hoever_552/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Nathan_Tella_1237/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Harry_Tyrer_1376/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jeffrey_Schlupp_214/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Alphonse_Areola_513/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Nick_Pope_413/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Kevin_Long_1340/gw.csv with own_team and gameweek.
Updated Fantasy

Updated Fantasy-Premier-League/data/2021-22/players/DEF_Ben_Nelson_616/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Bernardo_Veiga de Carvalho e Silva_342/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Willian_Borges da Silva_848/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Philip_Zinckernagel_1493/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Dynel_Simeu_1464/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Axel_Tuanzebe_282/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Maxwell_Haygarth_1320/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_John_McGinn_48/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Dwight_Gayle_1216/gw.csv with own_tea

Updated Fantasy-Premier-League/data/2021-22/players/DEF_Benjamin_White_24/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_João_Cavaco Cancelo_353/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Juan Camilo_Hernández Suárez_1509/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Oriol_Romeu Vidal_1233/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Ed_Turns_1097/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Owen_Beck_866/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Kamil_Conteh_1515/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Tyler_Dibling_608/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Matthew_Pollock_1506/gw.csv with own_team and gamewee

Updated Fantasy-Premier-League/data/2021-22/players/FWD_Danny_Ings_523/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Neal_Maupay_229/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Marcus_Rashford_385/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Marc_Navarro_1499/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Ben_Davies_484/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Reece_James_172/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Willy_Caballero_1244/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Ederson_Santana de Moraes_347/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Anwar_El Ghazi_1071/gw.csv with own_team and gameweek.
Updated Fant

...

Updated Fantasy-Premier-League/data/2024-25/players/MID_Georginio_Rutter_618/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Luke_Shaw_387/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Joe_Willock_420/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Deniz_Undav_143/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Levi_Colwill_162/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Facundo_Buonanotte_117/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Wesley_Fofana_189/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Michail_Antonio_512/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Phil_Foden_348/gw.csv with own_team and gameweek.
Updated Fantasy-Premi

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kevin_Mbabu_250/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Eddie_Nketiah_11/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Dango_Ouattara_74/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Pedro_Cardoso de Lima_561/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_James_Tarkowski_236/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Pierre-Emile_Højbjerg_490/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Alex_Mighten_435/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Dean_Huijsen_580/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Lewis_Dobbin_37/gw.csv with own_team and gameweek.
Updated 

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jannik_Vestergaard_307/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Rúben_Gato Alves Dias_361/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Christian_Nørgaard_101/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Ângelo Gabriel_Borges Damaceno_151/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Luis_Díaz_327/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jurriën_Timber_6/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Ross_Stewart_473/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Pascal_Groß_125/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Amadou_Onana_233/gw.csv with own_team and gam

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Valentín_Barco_116/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Neco_Williams_437/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Harvey_Davies_669/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Hannibal_Mejbri_373/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Daniel_Bentley_536/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Jaidon_Anthony_61/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Odeluga_Offiah_139/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Rodrigo_Bentancur_480/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Raheem_Sterling_186/gw.csv with own_team and gameweek.
Updated Fa

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jahmai_Simpson-Pusey_672/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Tariq_Lamptey_131/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Moussa_Diaby_34/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Sander_Berge_622/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kenny_Tete_257/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Amad_Diallo_364/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Daniel_Iversen_294/gw.csv with own_team and gameweek.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Daniel_Muñoz_211/gw.csv with own_team and gameweek.


In [14]:
# Now we can add short team names for player and opponent and also strengths of both teams from teams.csv
def add_team_strengths_and_short_names(data_directory, season, teams_file):
    """
    Add team strengths and short names for players and opponents in gw.csv files.

    Args:
        data_directory (str): Base directory containing the season data.
        season (str): Season name (e.g., "2022-23").
        teams_file (str): Name of the teams.csv file.
    """
    teams_path = os.path.join(data_directory, season, teams_file)
    if not check_file_exists(teams_path):
        return

    # Load teams.csv and index by team ID
    teams_data = load_csv(teams_path)
    if teams_data is None:
        return
    teams_data = teams_data.set_index("id")

    players_folder = os.path.join(data_directory, season, "players")
    if not check_file_exists(players_folder):
        return

    # Iterate through each player folder and update gw.csv
    for player_folder in os.listdir(players_folder):
        player_folder_path = os.path.join(players_folder, player_folder)
        gw_file_path = os.path.join(player_folder_path, "gw.csv")

        if check_file_exists(gw_file_path):
            try:
                gw_data = load_csv(gw_file_path)
                if gw_data is None:
                    continue

                # Merge opponent team data
                gw_data = gw_data.merge(
                    teams_data[[
                        'short_name', 'strength_attack_home', 'strength_attack_away',
                        'strength_defence_home', 'strength_defence_away'
                    ]],
                    left_on='opponent_team',
                    right_index=True,
                    how='left'
                ).rename(columns={
                    'short_name': 'opponent_short_name',
                    'strength_attack_home': 'opponent_attack_home',
                    'strength_attack_away': 'opponent_attack_away',
                    'strength_defence_home': 'opponent_defence_home',
                    'strength_defence_away': 'opponent_defence_away'
                })

                # Add opponent strengths
                gw_data['opponent_attack'] = gw_data.apply(
                    lambda x: x['opponent_attack_away'] if x['was_home'] else x['opponent_attack_home'], axis=1
                )
                gw_data['opponent_defense'] = gw_data.apply(
                    lambda x: x['opponent_defence_away'] if x['was_home'] else x['opponent_defence_home'], axis=1
                )

                # Merge own team data
                gw_data = gw_data.merge(
                    teams_data[[
                        'short_name', 'strength_attack_home', 'strength_attack_away',
                        'strength_defence_home', 'strength_defence_away'
                    ]],
                    left_on='own_team',
                    right_index=True,
                    how='left'
                ).rename(columns={
                    'short_name': 'own_short_name',
                    'strength_attack_home': 'own_attack_home',
                    'strength_attack_away': 'own_attack_away',
                    'strength_defence_home': 'own_defence_home',
                    'strength_defence_away': 'own_defence_away'
                })

                # Add own team strengths
                gw_data['own_attack'] = gw_data.apply(
                    lambda x: x['own_attack_home'] if x['was_home'] else x['own_attack_away'], axis=1
                )
                gw_data['own_defense'] = gw_data.apply(
                    lambda x: x['own_defence_home'] if x['was_home'] else x['own_defence_away'], axis=1
                )

                # Drop temporary columns
                gw_data.drop(columns=[
                    'opponent_attack_home', 'opponent_attack_away',
                    'opponent_defence_home', 'opponent_defence_away',
                    'own_attack_home', 'own_attack_away',
                    'own_defence_home', 'own_defence_away'
                ], inplace=True)

                # Save updated gw.csv
                save_csv(gw_data, gw_file_path)
                log(f"Updated {gw_file_path} with team strengths and short names.")
            except Exception as e:
                log(f"Error processing {gw_file_path}: {e}", level="ERROR")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2022-23", "2023-24", "2024-25"]
teams_file = "teams.csv"

for season in seasons:
    log(f"Processing season: {season}", level="INFO")
    add_team_strengths_and_short_names(data_directory, season, teams_file)

INFO: Processing season: 2022-23
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Jacob_Ramsey_53/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/MID_Jacob_Ramsey_53/gw.csv with team strengths and short names.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/GK_Marek_Rodák_839/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/GK_Marek_Rodák_839/gw.csv with team strengths and short names.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/FWD_Ellis_Simms_826/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/FWD_Ellis_Simms_826/gw.csv with team strengths and short names.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Sean_McAllister_829/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/MID_Sean_McAllister_829/gw.csv with team strengths and short names.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/FWD_Julián_Álvarez_352/gw.csv
INFO: Updated Fantasy-Premier-

Updated Fantasy-Premier-League/data/2020-21/players/FWD_Lys_Mousset_1673/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Isaac_Price_1130/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Morgan_Gibbs-White_433/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Max_Kilman_524/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Filip_Krovinovic_1720/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Kortney_Hause_40/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jeff_Hendrick_949/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Taylor_Harwood-Bellis_461/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/p

Updated Fantasy-Premier-League/data/2021-22/players/MID_Kaide_Gordon_867/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Ryan_Bertrand_1146/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Alex_Kral_1530/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Japhet_Tanganga_1034/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Steven_Bergwijn_1255/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_James_Milner_134/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Jóhann Berg_Gudmundsson_766/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Daniel_Chesters_1533/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-

Updated Fantasy-Premier-League/data/2021-22/players/GK_Robert_Sánchez_185/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Diogo_Dalot Teixeira_369/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Jürgen_Locadia_1333/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Martin_Kelly_1361/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Marcus_Bettinelli_155/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Conor_Coventry_1041/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Łukasz_Fabiański_521/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Charlie_Savage_1419/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/20

Updated Fantasy-Premier-League/data/2021-22/players/DEF_Owen_Beck_866/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Kamil_Conteh_1515/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Tyler_Dibling_608/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Matthew_Pollock_1506/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Shane_Duffy_1132/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Jesse_Lingard_1223/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Diego_Llorente_1167/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Andrew_Omobamidele_440/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2021-22/player

...

Updated Fantasy-Premier-League/data/2024-25/players/MID_Ibrahim_Sangaré_442/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Jack_Hinshelwood_126/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Tyrone_Mings_49/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_David_Ozoh_212/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Ryan_Fraser_397/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Pablo_Sarabia_565/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Abdul_Fatawu_570/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Imari_Samuels_663/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Fly

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Malo_Gusto_171/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Tom_Heaton_374/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Bruno_Borges Fernandes_366/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Abdoulaye_Doucouré_217/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Andreas_Hoelgebaum Pereira_240/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Sergio_Reguilón_496/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/GK_José_Malheiro de Sá_554/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Lewis_O'Brien_439/gw.csv with team strengths and short names.
Updated Fantasy-Premier-Leagu

Updated Fantasy-Premier-League/data/2024-25/players/MID_Pascal_Groß_125/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Amadou_Onana_233/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Brajan_Gruda_595/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Will_Smallbone_471/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Raúl_Jiménez_252/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Adam_Webster_147/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Caleb_Wiley_575/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Sékou_Mara_468/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Ilkay_Günd

Updated Fantasy-Premier-League/data/2024-25/players/MID_Tom_Cairney_243/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Harry_Wilson_259/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Tyler_Morton_331/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Sil_Swinkels_640/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Jordan_Ayew_192/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Valentín_Barco_116/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Neco_Williams_437/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Harvey_Davies_669/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Hanniba

Updated Fantasy-Premier-League/data/2024-25/players/GK_Asmir_Begovic_624/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Rico_Henry_92/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Manuel_Ugarte_652/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Diogo_Teixeira da Silva_317/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Malang_Sarr_176/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Rayan_Aït-Nouri_533/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Reiss_Nelson_10/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Alfie_Whiteman_510/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/D

In [15]:
# This isn't used for now as we use newer season data
# def add_crowds_to_gw(data_directory, season):
#     """
#     Add a 'crowds' column to gw.csv for each player to indicate if there were crowds at the stadium.

#     Args:
#         data_directory (str): Base directory containing the season data.
#         season (str): Season name (e.g., "2020-21").

#     Returns:
#         None
#     """
#     # Define the period without crowds (COVID restrictions)
#     no_crowds_start = datetime(2020, 3, 15, tzinfo=timezone.utc)
#     no_crowds_end = datetime(2021, 6, 17, tzinfo=timezone.utc)

#     players_folder = os.path.join(data_directory, season, "players")
#     if not check_file_exists(players_folder):
#         return

#     # Iterate through each player's folder
#     for player_folder in os.listdir(players_folder):
#         player_folder_path = os.path.join(players_folder, player_folder)
#         gw_file_path = os.path.join(player_folder_path, "gw.csv")

#         if check_file_exists(gw_file_path):
#             try:
#                 gw_data = load_csv(gw_file_path)
#                 if gw_data is None or "kickoff_time" not in gw_data.columns:
#                     log(f"'kickoff_time' not found in {gw_file_path}. Skipping.", level="WARNING")
#                     continue

#                 # Convert kickoff_time to UTC and add 'crowds' column
#                 gw_data['kickoff_time'] = pd.to_datetime(gw_data['kickoff_time']).dt.tz_convert('UTC')
#                 gw_data['crowds'] = gw_data['kickoff_time'].apply(
#                     lambda x: 0 if no_crowds_start <= x <= no_crowds_end else 1
#                 )

#                 save_csv(gw_data, gw_file_path)
#                 log(f"Updated {gw_file_path} with crowds column.")
#             except Exception as e:
#                 log(f"Error processing {gw_file_path}: {e}", level="ERROR")

# data_directory = "Fantasy-Premier-League/data"
# seasons = ["2022-23", "2023-24", "2024-25"]

# for season in seasons:
#     log(f"Processing season: {season}", level="INFO")
#     add_crowds_to_gw(data_directory, season)

Updated Fantasy-Premier-League/data/2021-22/players/FWD_Jamie_Vardy_306/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Jamal_Baptiste_1529/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Christian_Eriksen_370/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Jamie_Shackleton_1171/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Mads_Roerslev Rasmussen_105/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Matt_Ritchie_951/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Davinson_Sánchez_1033/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Allan_Marques Loureiro_1122/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Shandon_Baptiste_721/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_J

Updated Fantasy-Premier-League/data/2021-22/players/FWD_Anthony_Martial_931/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Tomáš_Souček_530/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Anthony_Gordon_398/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Lamare_Bogarde_598/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Nathan_Collins_88/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Anthony_Elanga_432/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Reece_Hannam_1363/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Remi_Matthews_209/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Michael_Obafemi_768/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_João Pedro_Junqueira de Jesus_

Updated Fantasy-Premier-League/data/2021-22/players/MID_Ismaïla_Sarr_585/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Kyle_Walker-Peters_476/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Jaden_Philogene_569/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Heung-Min_Son_1470/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Tom_Cleverley_1484/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Pascal_Struijk_1172/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Joe_White_419/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Luke_Cundle_540/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Josh_Dasilva_90/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Moise_Kean_1374/gw.csv with crowd

Updated Fantasy-Premier-League/data/2021-22/players/MID_Billy_Gilmour_124/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Milot_Rashica_1443/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Moussa_Sissoko_1478/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Alejandro_Garnacho_372/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Ryan_Sessegnon_579/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Przemyslaw_Placheta_1439/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Will_Norris_1346/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Andreas_Söndergaard_1539/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Jack_Jenkins_1390/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Rob_Holding_202

Updated Fantasy-Premier-League/data/2021-22/players/FWD_Edinson_Cavani_1408/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Joshua_Sargent_1448/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Joseph_McGlynn_1351/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Nampalys_Mendy_1148/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Serge_Aurier_963/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Samuel_Kalu_1519/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Scott_McTominay_381/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Salomón_Rondón_1121/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Adrián_San Miguel del Castillo_855/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Jack_Harr

...

Updated Fantasy-Premier-League/data/2022-23/players/MID_Máximo_Perrone_921/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Konstantinos_Tsimikas_337/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Ethan_Pinnock_104/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Vitaly_Janelt_94/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/GK_Martin_Dúbravka_396/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Ayoze_Pérez_1150/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Andrew_Robertson_335/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Federico_Fernández_1214/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Rodrigo 'Rodri'_Hernandez_360/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_James_

Updated Fantasy-Premier-League/data/2022-23/players/DEF_Conor_Coady_288/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Jakub_Moder_137/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/FWD_Danny_Welbeck_148/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/GK_Tom_Heaton_374/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Bruno_Borges Fernandes_366/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Abdoulaye_Doucouré_217/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Andreas_Hoelgebaum Pereira_240/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Sergio_Reguilón_496/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/GK_José_Malheiro de Sá_554/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Lewis_O'B

Updated Fantasy-Premier-League/data/2022-23/players/MID_Steven_Alzate_738/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Pascal_Groß_125/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Amadou_Onana_233/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Will_Smallbone_471/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/FWD_John-Kymani_Gordon_821/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/FWD_Raúl_Jiménez_252/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Lucas_Torreira di Pascua_1068/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Jordan_Zemura_1079/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Adam_Webster_147/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Darko_Gyabi_1183/gw.c

Updated Fantasy-Premier-League/data/2022-23/players/MID_Tom_Cairney_243/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Harry_Wilson_259/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Sil_Swinkels_640/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Jordan_Ayew_192/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Neco_Williams_437/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/GK_Harvey_Davies_669/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/GK_Daniel_Bentley_536/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Jaidon_Anthony_61/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Odeluga_Offiah_139/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Richie_Laryea_972/gw.csv with crowds colu

Updated Fantasy-Premier-League/data/2022-23/players/FWD_Terry_Ablade_1139/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Malang_Sarr_176/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Rayan_Aït-Nouri_533/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Reiss_Nelson_10/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Aaron_Cresswell_517/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/GK_George_Shelvey_981/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Trent_Alexander-Arnold_311/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/MID_Carlos_Alcaraz Durán_450/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/DEF_Calum_Chambers_689/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2022-23/players/GK_Oliwier_Zych_641/gw

In [16]:
# Let's add season number
def add_season_to_gw(data_directory, season):
    """
    Add the season year as a column to gw.csv files for each player.

    Args:
        data_directory (str): Base directory containing the season data.
        season (str): Season name (e.g., "2020-21").

    Returns:
        None
    """
    # Extract the start year from the season folder (e.g., "2021-22" -> "2021")
    try:
        season_year = int(season.split("-")[0])
    except ValueError:
        log(f"Invalid season format: {season}. Skipping.", level="ERROR")
        return

    players_folder = os.path.join(data_directory, season, "players")
    if not check_file_exists(players_folder):
        return

    for player_folder in os.listdir(players_folder):
        player_folder_path = os.path.join(players_folder, player_folder)
        gw_file_path = os.path.join(player_folder_path, "gw.csv")

        if check_file_exists(gw_file_path):
            try:
                gw_data = load_csv(gw_file_path)
                if gw_data is None:
                    continue

                gw_data["season"] = season_year

                save_csv(gw_data, gw_file_path)
                log(f"Updated {gw_file_path} with season column.")
            except Exception as e:
                log(f"Error processing {gw_file_path}: {e}", level="ERROR")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2022-23", "2023-24", "2024-25"]

for season in seasons:
    log(f"Processing season: {season}", level="INFO")
    add_season_to_gw(data_directory, season)

INFO: Processing season: 2022-23
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Jacob_Ramsey_53/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/MID_Jacob_Ramsey_53/gw.csv with season column.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/GK_Marek_Rodák_839/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/GK_Marek_Rodák_839/gw.csv with season column.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/FWD_Ellis_Simms_826/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/FWD_Ellis_Simms_826/gw.csv with season column.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/MID_Sean_McAllister_829/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/MID_Sean_McAllister_829/gw.csv with season column.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/players/FWD_Julián_Álvarez_352/gw.csv
INFO: Updated Fantasy-Premier-League/data/2022-23/players/FWD_Julián_Álvarez_352/gw.csv with seaso

In [17]:
# Finally - let's make csv with players from each position for each season
def process_season_for_positions(data_directory, season):
    """
    Process a single season to create position-specific CSVs for players.

    Args:
        data_directory (str): Base directory containing the season data.
        season (str): Season name (e.g., "2022-23").

    Returns:
        None
    """
    players_folder = os.path.join(data_directory, season, "players")
    if not check_file_exists(players_folder, log_missing=True):
        return

    processed_data_folder = os.path.join(data_directory, season, "processed_data")
    os.makedirs(processed_data_folder, exist_ok=True)

    # DataFrames for each position
    position_dfs = {
        "DEF": pd.DataFrame(),
        "MID": pd.DataFrame(),
        "FWD": pd.DataFrame(),
        "GK": pd.DataFrame(),
    }

    # Iterate through player directories
    for player_folder in os.listdir(players_folder):
        # Extract position from folder name
        position = player_folder.split("_")[0]

        # Read the player's gw.csv
        player_folder_path = os.path.join(players_folder, player_folder)
        gw_file_path = os.path.join(player_folder_path, "gw.csv")

        if check_file_exists(gw_file_path):
            try:
                # Load gw.csv and add it to the respective position DataFrame
                gw_data = load_csv(gw_file_path)
                if gw_data is not None and position in position_dfs:
                    position_dfs[position] = pd.concat(
                        [position_dfs[position], gw_data], ignore_index=True
                    )
            except Exception as e:
                log(f"Error processing {gw_file_path}: {e}", level="ERROR")

    # Save combined CSV for each position in the processed_data folder
    for position, df in position_dfs.items():
        if not df.empty:
            output_file_path = os.path.join(processed_data_folder, f"{position}_players.csv")
            save_csv(df, output_file_path)
            log(f"Saved {position} players for season {season} to {output_file_path}.")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2022-23", "2023-24", "2024-25"]

for season in seasons:
    log(f"Processing season: {season}", level="INFO")
    process_season_for_positions(data_directory, season)

INFO: Processing season: 2022-23
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/processed_data/DEF_players.csv
INFO: Saved DEF players for season 2022-23 to Fantasy-Premier-League/data/2022-23/processed_data/DEF_players.csv.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/processed_data/MID_players.csv
INFO: Saved MID players for season 2022-23 to Fantasy-Premier-League/data/2022-23/processed_data/MID_players.csv.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/processed_data/FWD_players.csv
INFO: Saved FWD players for season 2022-23 to Fantasy-Premier-League/data/2022-23/processed_data/FWD_players.csv.
INFO: Saved CSV: Fantasy-Premier-League/data/2022-23/processed_data/GK_players.csv
INFO: Saved GK players for season 2022-23 to Fantasy-Premier-League/data/2022-23/processed_data/GK_players.csv.
INFO: Processing season: 2023-24
INFO: Saved CSV: Fantasy-Premier-League/data/2023-24/processed_data/DEF_players.csv
INFO: Saved DEF players for season 2023-24 to Fantasy-Premier